## Contact-based metrics

Import files and build topology description

In [1]:
import numpy as np
import mdtraj as md
import prolintpy as pl

In [2]:
t = md.load('./data/test_data_1.xtc', top='./data/test_data_1.gro')

In [3]:
resolution = "martini"
combine_proteins = False
lipids = pl.Lipids(t.topology, resolution=resolution, lipid_names=["CHOL", "POPS"])
proteins = pl.Proteins(t.topology, resolution=resolution).system_proteins(merge=combine_proteins)

## Build a prolintpy.ComputeContacts object

We use this object for all contact calculations

In [4]:
contacts = pl.ComputeContacts(t, proteins, lipids)

In [5]:
contacts

Given a list of residues and a cutoff distance, calculate all neighboring lipids: 

In [6]:
residues = [*range(85, 130)]
cutoff = 0.5 # nm

In [7]:
# do not supply anything to residues argument in order for all residues to be included
result = contacts.compute_neighbors(t, residues=residues, cutoff=cutoff, grouped=False)

Working on protein copy: 0


The output is a nested dictionary: <br>
protein name > protein copy > residue ID > prolintpy.LPContacts object

In [8]:
result

{'Protein0': {0: {85: <prolintpy.LPContacts for residue 85>,
   86: <prolintpy.LPContacts for residue 86>,
   87: <prolintpy.LPContacts for residue 87>,
   88: <prolintpy.LPContacts for residue 88>,
   89: <prolintpy.LPContacts for residue 89>,
   90: <prolintpy.LPContacts for residue 90>,
   91: <prolintpy.LPContacts for residue 91>,
   92: <prolintpy.LPContacts for residue 92>,
   93: <prolintpy.LPContacts for residue 93>,
   94: <prolintpy.LPContacts for residue 94>,
   95: <prolintpy.LPContacts for residue 95>,
   96: <prolintpy.LPContacts for residue 96>,
   97: <prolintpy.LPContacts for residue 97>,
   98: <prolintpy.LPContacts for residue 98>,
   99: <prolintpy.LPContacts for residue 99>,
   100: <prolintpy.LPContacts for residue 100>,
   101: <prolintpy.LPContacts for residue 101>,
   102: <prolintpy.LPContacts for residue 102>,
   103: <prolintpy.LPContacts for residue 103>,
   104: <prolintpy.LPContacts for residue 104>,
   105: <prolintpy.LPContacts for residue 105>,
   106:

Extract contact data for a particular residue

In [9]:
print (result['Protein0'][0][85])

<prolintpy.LPContacts for residue 85>


In [10]:
contact_r85 = result['Protein0'][0][88]

In [11]:
contact_r85

<prolintpy.LPContacts for residue 88>

Not all interactions are equal! Residue 88 interacts with only one cholesterol lipid during the length of the input trajectory. However, it forms several interactions with POPS lipids, even though the input test trajectory is quite short. <br>The output is a dictionary with lipids as keys and the contact duration as dictionary values. The time units here are the same as MDTraj. 

In [12]:
contact_r85.contacts

{'POPS': [300000.0, 900000.0, 300000.0, 600000.0, 1200000.0, 300000.0],
 'CHOL': [3900000.0]}

We can also retrieve the identity of the lipids that form the interactions above. <br>
This is very useful if we want to build custom metrics or just in general customize the workflow

In [13]:
contact_r85.lipids

{'POPS': array([1677, 1773, 1817, 1888, 1889, 1934]), 'CHOL': array([2951])}

Occupancy is a binary measure so we need to retrieve it separately

In [14]:
contact_r85.occupancy

{'POPS': array([1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1.]),
 'CHOL': array([1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.])}

Supplying the mdtraj argument to `contacts.compute_neighbors()` will result in the whole MDTraj output being saved to `contact_r85` above, however, the when used it will result in very large object sizes and currently has been disabled. We are planning on removing this option from the source code, but if you would like to revert back this functionality please have a look here: https://github.com/bisejdiu/prolint/blob/main/prolint/core/computecontacts.py#L155

In [15]:
# Does not work because it has been disabled. 
# contact_r85.mdtraj

### Customizability 

Altogether, the `prolintpy` capabilities highlighted above make it clear how easy it is to customize the workflow to your use-case. This is because analysis is not a closed system, and `prolintpy` provides access 
to its internal data at every step of the way. You can use the `result` dictionary above and loop through the different options without worrying about any of the other features of `prolintpy`. 

Nevertheless, `prolintpy` provides several helper functions and features which make working with the contact informations above really easy

### Helper functions 

Retrieve contact information for all lipids in the system

In [16]:
pl.retrieve_contacts(result, 88, contacts='contacts')

defaultdict(list,
            {'POPS': [array([ 300000.,  900000.,  300000.,  600000., 1200000.,  300000.])],
             'CHOL': [array([3900000.])]})

In [17]:
pl.retrieve_contacts(result, 88, contacts='occupancy')

defaultdict(list,
            {'POPS': [array([1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1.])],
             'CHOL': [array([1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.])]})

Retrieve contact information for a specific lipid in the system

In [18]:
pl.retrieve_contacts_flat(result, 88, lipid="CHOL", contacts='contacts')

array([3900000.])

In [19]:
pl.retrieve_contacts_flat(result, 88, lipid="POPS", contacts='occupancy')

array([1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1.])

### Build a pandas DataFrame

pandas DataFrame are the most convenient way to manipulate contact information. <br>
Building a DataFrame using `prolintpy` is a straightforward process. 

In [20]:
df = pl.contacts_dataframe(result, proteins, t, radius=cutoff)

In [21]:
df.head()

,Mean_Duration,Longest_Duration,Sum_of_all_Contacts,Lipid_Number,Normalized_Lipid_Number,Occupancy,Protein,Lipids,Radius,ResID,ResName
0,0.0,0.0,0.0,0.0000,0.0000,0.000000,Protein0,CHOL,0.5,85,ARG
1,0.0,0.0,0.0,0.0000,0.0000,0.000000,Protein0,CHOL,0.5,86,GLN
2,0.3,0.3,1.2,0.2500,0.2500,23.529412,Protein0,CHOL,0.5,87,ARG
3,3.9,3.9,3.9,0.8125,0.8125,76.470588,Protein0,CHOL,0.5,88,TYR
4,0.0,0.0,0.0,0.0000,0.0000,0.000000,Protein0,CHOL,0.5,89,MET


Sort by the most interacting residues in terms of the Longest_Duration metric

In [22]:
df.sort_values('Longest_Duration', ascending=False).head()

,Mean_Duration,Longest_Duration,Sum_of_all_Contacts,Lipid_Number,Normalized_Lipid_Number,Occupancy,Protein,Lipids,Radius,ResID,ResName
6,1.70,4.5,5.1,1.0625,1.0625,88.235294,Protein0,CHOL,0.5,91,LYS
3,3.90,3.9,3.9,0.8125,0.8125,76.470588,Protein0,CHOL,0.5,88,TYR
43,1.95,3.6,3.9,0.8125,0.8125,70.588235,Protein0,CHOL,0.5,128,PHE
36,1.50,3.0,4.5,0.9375,0.9375,64.705882,Protein0,CHOL,0.5,121,TRP
40,1.95,3.0,3.9,0.8125,0.8125,58.823529,Protein0,CHOL,0.5,125,LEU


Get the residue ids and indices of the top 5 residues interacting with cholesterol as measured by the Longest_Duration metric

In [23]:
top_residues = df[df.Lipids == "CHOL"].sort_values('Longest_Duration', ascending=False).ResID.to_list()[:10]
print ("Most interacting residues are: ", top_residues, end="\n" + '~' * 80)

Most interacting residues are:  [91, 88, 128, 121, 125, 124, 117, 118, 122, 114]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [24]:
[f'Residue {res} with indices: {proteins[0].get_indices([res], suppress=True)[0]}' for res in top_residues]

['Residue 91 with indices: [225 226 227 228]',
 'Residue 88 with indices: [220 221]',
 'Residue 128 with indices: [304 305]',
 'Residue 121 with indices: [292 293]',
 'Residue 125 with indices: [299 300]',
 'Residue 124 with indices: [297 298]',
 'Residue 117 with indices: [284 285]',
 'Residue 118 with indices: [286 287]',
 'Residue 122 with indices: [294 295]',
 'Residue 114 with indices: [279 280]']